## Regression - Diabetes [numpy]

- created: 2024.11.17

### Data

In [1]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data
x, y = load_diabetes(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

print(x_train_scaled.shape, y_train.shape)
print(x_test_scaled.shape, y_test.shape)

(353, 10) (353,)
(89, 10) (89,)


### [scikit-learn] Modeling and Training

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# sklearn model
model = LinearRegression()
model.fit(x_train_scaled, y_train)

# Training
y_pred = model.predict(x_train_scaled)
acc = r2_score(y_train, y_pred)
print(f'Train Accuracy: {acc:.4f}')

# Evaluation
y_pred = model.predict(x_test_scaled)
acc = r2_score(y_test, y_pred)
print(f'Test  Accuracy: {acc:.4f}')

Train Accuracy: 0.5279
Test  Accuracy: 0.4526


### [numpy] Modeling and Training

- Manual backward propagation
- Manual update of weights and biases

In [3]:
import numpy as np
from scipy.special import expit as sigmoid

def r2_score(y_pred, y_true):
    mean_y_true = np.mean(y_true)
    ss_tot = np.sum((y_true - mean_y_true) ** 2)
    ss_res = np.sum((y_true - y_pred) ** 2)
    return 1 - (ss_res / ss_tot)

## Data
x, y = x_train_scaled, y_train.reshape(-1, 1)
print(x.shape, y.shape)

(353, 10) (353, 1)


In [4]:
## Model: 2-layer MLP
np.random.seed(42)
input_size, hidden_size, output_size = 10, 100, 1

w1 = np.random.randn(input_size, hidden_size)   # weight of 1st layer
b1 = np.zeros(hidden_size)                      # bias of 1st layer
w2 = np.random.randn(hidden_size, output_size)  # weight of 2nd layer
b2 = np.zeros(output_size)                      # bias of 2nd layer

## Train
n_epochs = 10000
learning_rate = 0.1

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    z1 = np.dot(x, w1) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(a1, w2) + b2
    out = z2

    loss = np.mean((out - y)**2)
    score = r2_score(out, y)

    # Backward progapation
    grad_out = 2 * (out - y) / y.shape[0]
    grad_z2 = grad_out
    grad_w2 = np.dot(a1.T, grad_z2)
    grad_b2 = np.sum(grad_z2, axis=0)

    grad_a1 = np.dot(grad_z2, w2.T)
    grad_z1 = a1 * (1 - a1) * grad_a1
    grad_w1 = np.dot(x.T, grad_z1)
    grad_b1 = np.sum(grad_z1, axis=0)

    # Update weights and biases
    w1 -= learning_rate * grad_w1
    b1 -= learning_rate * grad_b1
    w2 -= learning_rate * grad_w2
    b2 -= learning_rate * grad_b2

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score:.4f}")

[1000/10000] loss: 2143.72 score: 0.6472
[2000/10000] loss: 2048.47 score: 0.6629
[3000/10000] loss: 2030.28 score: 0.6659
[4000/10000] loss: 2000.32 score: 0.6708
[5000/10000] loss: 1965.99 score: 0.6765
[6000/10000] loss: 1964.31 score: 0.6767
[7000/10000] loss: 1873.52 score: 0.6917
[8000/10000] loss: 1782.18 score: 0.7067
[9000/10000] loss: 1756.15 score: 0.7110
[10000/10000] loss: 1746.60 score: 0.7126
